# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [44]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [45]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [46]:
# print names of all countries
root = document_tree.getroot()
for child in root:
    name = child.find('name').text
    print (name)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [47]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':'),
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

### Exercise 1: 10 countries with lowest infant mortality rates

In [48]:
document = ET.parse( './data/mondial_database.xml' )

In [49]:
import numpy as np
import pandas as pd

In [90]:
#extracting xml data into dictionary, constructing a dataframe out of the dictionary and sorting by infant mortality
underlying_dict = {}
for country in document.iterfind('country'):
    name = country.find('name').text
    infant_mortality = country.find('infant_mortality')
    if infant_mortality is not None: 
        underlying_dict[name]=float(infant_mortality.text)
        
df = pd.DataFrame.from_dict(underlying_dict, orient = 'index')
df.columns=['infant mortality']
df.sort_values('infant mortality').head(10)

,infant mortality
Monaco,1.81
Japan,2.13
Norway,2.48
Bermuda,2.48
Singapore,2.53
Sweden,2.60
Czech Republic,2.63
Hong Kong,2.73
Macao,3.13
Iceland,3.15


### Exercise 2: 10 cities with the largest population

In [94]:
#constructing an empty dataframe for the required variables
population_df = pd.DataFrame(columns = ['country','city','population', 'year'])

#building four separate lists for countries, cities, population, and census year to populate our dataframe
countries = []
cities = []
populations = []
years = []

for country in document.iterfind('country'):
    country_name = country.find('name').text
    for city in country.iter('city'):
        city_name = city.find('name').text
        for population in city.iterfind('population'):
            population_var = int(population.text)
            year = population.attrib['year']
            countries.append(country_name)
            cities.append(city_name)
            populations.append(population_var)
            years.append(year)

population_df = pd.DataFrame({
        'country': countries,
        'city': cities,
        'population': populations,
        'year': years
    })

#multiple cities have multiple census years/population counts. Constructing a function to extract only the highest population count for each city
def my_agg(group_df):
    max_population = group_df.population.max()
    return group_df[group_df['population'] == max_population]

#extracting final results: 10 largest cities by population count with the highest count (year of the count specified) 
population_gb = population_df.groupby(['city']).apply(my_agg)
population_gb.sort_values('population', ascending = False).head(10).reset_index(drop = True)

,city,country,population,year
0,Shanghai,China,22315474,2010
1,Istanbul,Turkey,13710512,2012
2,Delhi,India,12877470,2001
3,Mumbai,India,12442373,2011
4,Moskva,Russia,11979529,2013
5,Beijing,China,11716620,2010
6,São Paulo,Brazil,11152344,2010
7,Tianjin,China,11090314,2010
8,Guangzhou,China,11071424,2010
9,Shenzhen,China,10358381,2010


### Exercise 3: 10 largest ethnic groups by population across all countries

In [95]:
#constructing empty dataframe to hold results + extracting xml data into lists to populate the dataframe
ethnicgroup_df = pd.DataFrame(columns = ['country','population', 'year', 'ethnic group', 'percentage'])
countries_eg = []
population_eg = []
years_eg = []
ethnic_groups = []
percentage_eg = []

for country in document.iterfind('country'):
    country_name_eg = country.find('name').text
    for ethnic_group in country.iter('ethnicgroup'):
        ethnicgroup_name = ethnic_group.text
        ethnicgroup_percent = float(ethnic_group.attrib['percentage'])
        for population in country.iterfind('population'):
            population_num = int(population.text)
            year_num = int(population.attrib['year'])            
            
            countries_eg.append(country_name_eg)
            population_eg.append(population_num)
            years_eg.append(year_num)
            ethnic_groups.append(ethnicgroup_name)
            percentage_eg.append(ethnicgroup_percent)
            
ethnicgroup_df = pd.DataFrame ({
        'country':countries_eg,
        'population':population_eg,
        'year': years_eg,
        'ethnic group': ethnic_groups,
        'percentage': percentage_eg
    })

#population for each ethnic group is not given. We need to calculate it using the percentage for the ethnic group provided for each country
ethnicgroup_df['ethnic population count'] = (ethnicgroup_df.population * ethnicgroup_df.percentage)/100
ethnicgroup_df = ethnicgroup_df.round({'ethnic population count':0})
del ethnicgroup_df['population']
del ethnicgroup_df['percentage']

#defining function similar to the one in #2. This time we want to extract the latest census year for each population count
def my_agg(group_df):
     max_year = group_df.year.max()
     return group_df[group_df['year'] == max_year]

#grouping ethnic groups by country while applying the function to get latest population count for each ethnic group
ethnicgroup_gb = ethnicgroup_df.groupby(['country']).apply(my_agg)
del ethnicgroup_gb['year']

#summing up population count for each ethnic group and sorting in descending order
ethnicgroup_gb.groupby('ethnic group').sum().sort_values(['ethnic population count'], ascending = False).head(10)

,ethnic population count
ethnic group,
Han Chinese,1.245059e+09
Indo-Aryan,8.718156e+08
European,4.948722e+08
African,3.183251e+08
Dravidian,3.027137e+08
Mestizo,1.577344e+08
Bengali,1.467769e+08
Russian,1.318570e+08
Japanese,1.265342e+08


### 4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

#### a)

In [98]:
#building a dataframe of country codes and corresponding country names
countries = []
codes = []

for country in document.iterfind('country'):
    country_name = country.find('name').text
    country_code = country.attrib['car_code']
    
    countries.append(country_name)
    codes.append(country_code)
    
country_code_df = pd.DataFrame(columns = ['country','code'])
country_code_df = pd.DataFrame({
                    'country': countries,
                    'code': codes
    })  

#constructing a dataframe of rivers, their lengths, and associated country codes
rivers = []
length_river = []
rivers_country_code = []

for river in document.iterfind('river'):
    river_name = river.find('name').text
    
#some rivers are located in more than one country. <located> sub-element, if available, will thus provide the first country in which the river is located.   
    if river.find('located') is not None:
        river_cc = river.find('located').attrib['country']
    else:
        river_cc = river.attrib['country']
        
    len_var = river.find('length')
    if len_var is not None:
        len_num = float(len_var.text)
    
    rivers.append(river_name)
    length_river.append(len_num)
    rivers_country_code.append(river_cc)
    
rivers_df = pd.DataFrame(columns = ['river name', 'length', 'code'])
rivers_df = pd.DataFrame({
        'river name': rivers,
        'length': length_river,
        'code': rivers_country_code        
    })

#merging country code dataframe and rivers dataframe to get the full name of each country for a given river
final_river_df = country_code_df.merge(rivers_df)

#determining the longest river and outputting results
max_length = max(final_river_df['length'])
final_river_df = final_river_df[['country', 'river name', 'length']]
final_river_df[final_river_df.length == max_length]

,country,river name,length
168,Colombia,Amazonas,6448.0


In [99]:
#building a dataframe of lakes, their areas, and corresponding country codes
lakes = []
area_lakes = []
lakes_country_code = []

for lake in document.iterfind('lake'):
    lake_name = lake.find('name').text
    
#some lakes are located in more than one country. Similarly to the rivers dataframe above, we are using the <located> sub-element to get the first country on the list
    if lake.find('located') is not None:
        lake_cc = lake.find('located').attrib['country']
    else:
        lake_cc = lake.attrib['country']
    
    area_var = lake.find('area')
    if area_var is not None:
        area_num = float(area_var.text)
    
    lakes.append(lake_name)
    area_lakes.append(area_num)
    lakes_country_code.append(lake_cc)
    
lakes_df = pd.DataFrame(columns = ['lake name', 'area', 'code'])
lakes_df = pd.DataFrame({
        'lake name': lakes,
        'area': area_lakes,
        'code': lakes_country_code        
    })

#merging lakes dataframe with the country code dataframe to get lakes and their full country names
final_lakes_df = country_code_df.merge(lakes_df)

#identifying largest river
max_area = max(final_lakes_df.area)
final_lakes_df = final_lakes_df[['country', 'lake name', 'area']]
final_lakes_df[final_lakes_df.area == max_area]

,country,lake name,area
31,Russia,Caspian Sea,386400.0


In [100]:
#building a dataframe of airports, their elevations, and corresponding country codes. Similarly to a) and b) merging the dataframe with country code dataframe to get full country names
airports = []
airport_elevation = []
airport_country_code = []

for airport in document.iterfind('airport'):
    airport_name = airport.find('name').text
    airport_cc = airport.attrib['country']
    el_var = airport.find('elevation')
    if (el_var is not None) and el_var.text is not None:
        el_num = float(el_var.text)
    
    airports.append(airport_name)
    airport_elevation.append(el_num)
    airport_country_code.append(airport_cc)
    
airports_df = pd.DataFrame(columns = ['airport name', 'elevation', 'code'])
airports_df = pd.DataFrame({
        'airport name': airports,
        'elevation': airport_elevation,
        'code': airport_country_code        
    })

final_airports_df = country_code_df.merge(airports_df)

#identifying highest elevation
max_elevation = max(final_airports_df.elevation)
final_airports_df = final_airports_df[['country', 'airport name', 'elevation']]
final_airports_df[final_airports_df.elevation == max_elevation]

,country,airport name,elevation
1085,Bolivia,El Alto Intl,4063.0
